In [1]:
from italian_csv_type_prediction.dataframe_generators import SimpleDatasetGenerator
from italian_csv_type_prediction.models import TypePredictor
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
from multiprocessing import Pool, cpu_count
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [2]:
def dataset_generation(number:int):
    return SimpleDatasetGenerator(verbose=False).build(number, verbose=False)

def _dataset_generation(args):
    return dataset_generation(*args)

def parallel_dataset_generation(number:int):
    processes = min(cpu_count(), number)
    with Pool(cpu_count()) as p:
        Xs, ys = list(zip(*tqdm(
            p.imap(_dataset_generation, (
                (number//processes, )
                for _ in range(processes)
            )),
            desc="Creating dataset",
            total=processes,
            leave=False
        )))
    return np.vstack(Xs), np.concatenate(ys)

In [3]:
x_train, y_train = parallel_dataset_generation(1000)
x_test, y_test = parallel_dataset_generation(1000)

In [4]:
model = TypePredictor()

model.fit(x_train, y_train)

In [5]:
y_pred = model._model.predict(x_test)
y_train_pred = model._model.predict(x_train)

In [6]:
accuracy_score(y_test, y_pred), balanced_accuracy_score(y_test, y_pred)

(0.9986683055158879, 0.9986138130656635)

In [7]:
accuracy_score(y_train, y_train_pred), balanced_accuracy_score(y_train, y_train_pred)

(0.9995325475385223, 0.9995423677257853)

In [8]:
X, y = SimpleDatasetGenerator().generate_simple_dataframe(max_rows=10)

In [9]:
y

,ItalianFiscalCode,ItalianVAT,CadastreCode,Document,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,Region,...,Country,CountryCode,Name,Surname,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean
0,ItalianFiscalCode,NaN,CadastreCode,Document,Tax,Plate,NaN,NaN,ProvinceCode,Region,...,Country,NaN,NaN,NaN,String,EMail,NaN,Date,BiologicalSex,Boolean
1,ItalianFiscalCode,ItalianVAT,NaN,Document,NaN,Plate,NaN,ItalianZIPCode,ProvinceCode,Region,...,Country,CountryCode,Name,Surname,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean
2,ItalianFiscalCode,ItalianVAT,CadastreCode,Document,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,Region,...,NaN,NaN,NaN,NaN,NaN,EMail,PhoneNumber,NaN,BiologicalSex,Boolean
3,ItalianFiscalCode,ItalianVAT,NaN,Document,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,NaN,...,Country,CountryCode,Name,Surname,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean
4,ItalianFiscalCode,ItalianVAT,CadastreCode,NaN,Tax,Plate,Address,NaN,ProvinceCode,Region,...,NaN,NaN,Name,Surname,String,NaN,NaN,Date,BiologicalSex,Boolean
5,ItalianFiscalCode,ItalianVAT,CadastreCode,Document,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,Region,...,Country,CountryCode,NaN,Surname,String,EMail,NaN,Error,NaN,Boolean
6,ItalianFiscalCode,NaN,CadastreCode,Document,Tax,NaN,Address,ItalianZIPCode,ProvinceCode,Region,...,NaN,CountryCode,Name,Surname,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean
7,NaN,ItalianVAT,CadastreCode,Document,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,Region,...,Country,NaN,Name,Surname,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean
8,NaN,ItalianVAT,CadastreCode,NaN,NaN,Plate,Address,ItalianZIPCode,ProvinceCode,Region,...,Country,CountryCode,Name,NaN,String,EMail,PhoneNumber,Date,BiologicalSex,NaN
9,NaN,ItalianVAT,CadastreCode,Document,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,Region,...,NaN,CountryCode,Name,NaN,String,EMail,PhoneNumber,Date,NaN,Boolean


In [10]:
model.predict_dataframe(X)

,ItalianFiscalCode,ItalianVAT,CadastreCode,Document,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,Region,...,Country,CountryCode,Name,Surname,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean
0,ItalianFiscalCode,NaN,CadastreCode,Document,Tax,Plate,NaN,NaN,ProvinceCode,Region,...,Country,NaN,NaN,NaN,String,EMail,NaN,Date,BiologicalSex,Boolean
1,ItalianFiscalCode,ItalianVAT,NaN,Document,NaN,Plate,NaN,ItalianZIPCode,ProvinceCode,Region,...,Country,CountryCode,Name,Surname,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean
2,ItalianFiscalCode,ItalianVAT,CadastreCode,Document,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,Region,...,NaN,NaN,NaN,NaN,NaN,EMail,PhoneNumber,NaN,BiologicalSex,Boolean
3,ItalianFiscalCode,ItalianVAT,NaN,Document,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,NaN,...,Country,CountryCode,Name,Surname,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean
4,ItalianFiscalCode,ItalianVAT,CadastreCode,NaN,Tax,Plate,Address,NaN,ProvinceCode,Region,...,NaN,NaN,Name,Surname,String,NaN,NaN,Date,BiologicalSex,Boolean
5,ItalianFiscalCode,ItalianVAT,CadastreCode,Document,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,Region,...,Country,CountryCode,NaN,Surname,String,EMail,NaN,Error,NaN,Boolean
6,ItalianFiscalCode,NaN,CadastreCode,Document,Tax,NaN,Address,ItalianZIPCode,ProvinceCode,Region,...,NaN,CountryCode,Name,Surname,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean
7,NaN,ItalianVAT,CadastreCode,Document,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,Region,...,Country,NaN,Name,Surname,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean
8,NaN,ItalianVAT,CadastreCode,NaN,NaN,Plate,Address,ItalianZIPCode,ProvinceCode,Region,...,Country,CountryCode,Name,NaN,String,EMail,PhoneNumber,Date,BiologicalSex,NaN
9,NaN,ItalianVAT,CadastreCode,Document,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,Region,...,NaN,CountryCode,Name,NaN,String,EMail,PhoneNumber,Date,NaN,Boolean


In [11]:
from collections import Counter

mask = y_test != y_pred

true_labels = model._embedder._encoder.inverse_transform(y_test[mask])
predicted_labels = model._embedder._encoder.inverse_transform(y_pred[mask])

Counter(zip(true_labels, predicted_labels))

Counter({('Error', 'Document'): 174,
         ('CountryCode', 'NaN'): 73,
         ('Error', 'CountryCode'): 12,
         ('String', 'Error'): 151,
         ('Error', 'ItalianVAT'): 29,
         ('NaN', 'CountryCode'): 95,
         ('Error', 'Plate'): 1,
         ('Error', 'ProvinceCode'): 21,
         ('Error', 'ItalianZIPCode'): 8,
         ('Error', 'Name'): 10,
         ('Region', 'Error'): 3,
         ('Error', 'Tax'): 2,
         ('CountryCode', 'ProvinceCode'): 23,
         ('Error', 'NaN'): 1,
         ('Float', 'Error'): 8,
         ('CountryCode', 'Error'): 6,
         ('BiologicalSex', 'Error'): 3,
         ('ProvinceCode', 'Error'): 11,
         ('Document', 'Error'): 40,
         ('Error', 'String'): 29,
         ('Error', 'ItalianFiscalCode'): 18,
         ('Date', 'Error'): 2,
         ('Error', 'Date'): 4,
         ('Surname', 'String'): 5,
         ('PhoneNumber', 'Error'): 5,
         ('Municipality', 'Error'): 6,
         ('NaN', 'Error'): 13,
         ('EMail', 'Err